In [10]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('listings-2.csv')

In [4]:
display(df.head(3))

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,13913,https://www.airbnb.com/rooms/13913,20221210143044,2022-12-11,city scrape,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,54730,...,4.84,4.71,4.71,NaN,f,2,1,1,0,0.27
1,15400,https://www.airbnb.com/rooms/15400,20221210143044,2022-12-12,city scrape,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,https://a0.muscache.com/pictures/428392/462d26...,60302,...,4.83,4.93,4.74,NaN,f,1,1,0,0,0.58
2,172811,https://www.airbnb.com/rooms/172811,20221210143044,2022-12-10,previous scrape,Nice double bedroom in NW London,<b>The space</b><br />Nice double bedroom in N...,NaN,https://a0.muscache.com/pictures/1095093/0031f...,806983,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN


In [43]:
df_id = df[['id']]
df_id = df_id.rename(columns={"id": "image_id"})
display(df_id)

,image_id
0,13913
1,15400
2,172811
3,173082
4,42010
...,...
71933,778043744938747948
71934,778047302711995594
71935,778048561836095146
71936,778053470563773134


In [39]:
display((df_id['image_id'][71933]))

778043744938747948

In [11]:
with open('Interior_Mobile.json') as f:
    interior = json.load(f)
with open('Entire_Mobile.json') as f:
    entire = json.load(f)

In [32]:
print(interior[1])

{'image_id': '10007608', 'mean_score_prediction': 5.307502956231474}


In [31]:
print(type(interior[1]['mean_score_prediction']))

<class 'float'>


In [45]:
id_to_prediction = {item['image_id']: item['mean_score_prediction'] for item in interior}
df_id['interior'] = df_id['image_id'].apply(lambda x: id_to_prediction.get(str(x), float('nan')))


In [49]:
df_id = df_id.dropna()
display(df_id)

,image_id,interior
0,13913,5.221664
1,15400,5.099636
2,172811,4.972842
3,173082,5.765924
4,42010,5.487355
...,...,...
71192,773076895551829164,5.492463
71194,774435894636294510,5.150191
71195,773606537478247748,5.165789
71196,774439882794564032,4.964971


In [50]:
id_to_prediction_2 = {item['image_id']: item['mean_score_prediction'] for item in entire}
df_id['entire'] = df_id['image_id'].apply(lambda x: id_to_prediction_2.get(str(x), float('nan')))

In [51]:
display(df_id)

,image_id,interior,entire
0,13913,5.221664,5.358303
1,15400,5.099636,5.424931
2,172811,4.972842,3.823302
3,173082,5.765924,5.875356
4,42010,5.487355,5.249705
...,...,...,...
71192,773076895551829164,5.492463,5.812972
71194,774435894636294510,5.150191,4.793989
71195,773606537478247748,5.165789,5.690618
71196,774439882794564032,4.964971,4.758211


In [53]:
df_id['avg'] = (df_id['interior']+df_id['entire'])/2

In [54]:
display(df_id)

,image_id,interior,entire,avg
0,13913,5.221664,5.358303,5.289983
1,15400,5.099636,5.424931,5.262284
2,172811,4.972842,3.823302,4.398072
3,173082,5.765924,5.875356,5.820640
4,42010,5.487355,5.249705,5.368530
...,...,...,...,...
71192,773076895551829164,5.492463,5.812972,5.652717
71194,774435894636294510,5.150191,4.793989,4.972090
71195,773606537478247748,5.165789,5.690618,5.428203
71196,774439882794564032,4.964971,4.758211,4.861591


In [55]:
df_id.to_csv('id_score.csv',index=False)

In [64]:


# Assuming 'df' is a pandas dataframe with the column named 'values'
min_val = df_id['avg'].min()
max_val = df_id['avg'].max()

# Min-Max normalization formula
df_id['normalized_avg'] = ((df_id['avg'] - min_val) / (max_val - min_val)) * 9 + 1

# Print the updated dataframe
display(df_id)

,image_id,interior,entire,avg,normalized_avg
0,13913,5.221664,5.358303,5.289983,6.426101
1,15400,5.099636,5.424931,5.262284,6.300351
2,172811,4.972842,3.823302,4.398072,2.377051
3,173082,5.765924,5.875356,5.820640,8.835144
4,42010,5.487355,5.249705,5.368530,6.782684
...,...,...,...,...,...
71192,773076895551829164,5.492463,5.812972,5.652717,8.072820
71194,774435894636294510,5.150191,4.793989,4.972090,4.982947
71195,773606537478247748,5.165789,5.690618,5.428203,7.053583
71196,774439882794564032,4.964971,4.758211,4.861591,4.481308


In [63]:
df_id.to_csv('id_score.csv',index=False)

In [68]:
display(df_id[df_id['avg']==max_val])

,image_id,interior,entire,avg,normalized_avg
20011,20362814,5.596716,6.557745,6.07723,10.0
